In [1]:
!python --version

Python 3.10.10


In [2]:
import os
import cv2
import torch
import random
import sys
import pandas as pd
import numpy as np
from tqdm import tqdm
from pathlib import Path
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms
sys.path.append('/kaggle/input')
from myfinaldataset.superpoint import SuperPoint
from myfinaldataset.matching import Matching


def load_image_gray(path):
    image = cv2.imread(str(path), cv2.IMREAD_GRAYSCALE)
    image = image.astype(np.float32) / 255.
    resized = cv2.resize(image, (160, 120))
    return torch.tensor(resized).unsqueeze(0)


def parse_matrix(s):
    return np.array([float(x) for x in s.split(';')]).reshape(3, 3)


def parse_vector(s):
    return np.array([float(x) for x in s.split(';')]).reshape(3, 1)


class ImagePairDataset(Dataset):
    def __init__(self, csv_path, root_dir):
        self.data = pd.read_csv(csv_path)
        self.root_dir = Path(root_dir)
        self.groups = self.data.groupby("scene")
        self.scene_keys = list(self.groups.groups.keys())

    def __len__(self):
        return len(self.scene_keys)

    def __getitem__(self, idx):
        group = self.groups.get_group(self.scene_keys[idx])
        rows = group.sample(2)
        imgA_path = self.root_dir / rows.iloc[0]['image_path']
        imgB_path = self.root_dir / rows.iloc[1]['image_path']

        imgA = load_image_gray(imgA_path)
        imgB = load_image_gray(imgB_path)

        R = parse_matrix(rows.iloc[1]['rotation_matrix']).astype(np.float32)
        t = parse_vector(rows.iloc[1]['translation_vector']).astype(np.float32)

        return {
            'imageA': imgA,
            'imageB': imgB,
            'rotation': torch.tensor(R),
            'translation': torch.tensor(t)
        }


matching_config = {
    'superpoint': {
        'nms_radius': 3,
        'keypoint_threshold': 0.005,
        'max_keypoints': -1
    },
    'superglue': {
        'weights': 'outdoor',
        'sinkhorn_iterations': 20,
        'match_threshold': 0.2,
    }
}
matcher = Matching(matching_config)
matcher = matcher.eval()


def compute_loss_match(matching_result):
    if 'matches0' not in matching_result or 'matching_scores0' not in matching_result:
        return torch.tensor(1.0, requires_grad=True, device='cuda' if torch.cuda.is_available() else 'cpu')
    matches = matching_result['matches0'][0]  # [N]
    valid = matches > -1
    scores = matching_result['matching_scores0'][0][valid]
    if len(scores) == 0:
        return torch.tensor(1.0, device=matches.device, requires_grad=True)
    return (1.0 - scores.mean())


def run_epoch(model, dataloader, optimizer, device, train=True):
    model.train() if train else model.eval()
    total_loss = 0
    loop = tqdm(dataloader, desc='Train' if train else 'Val', leave=False)

    with torch.set_grad_enabled(train):
        for batch in loop:
            imageA = batch['imageA'].to(device, non_blocking=True)
            imageB = batch['imageB'].to(device, non_blocking=True)

            sp_outputA = model({'image': imageA})
            sp_outputB = model({'image': imageB})

            pred = matcher({
                'image0': imageA,
                'image1': imageB,
                'keypoints0': sp_outputA['keypoints'],
                'keypoints1': sp_outputB['keypoints'],
                'scores0': sp_outputA['scores'],
                'scores1': sp_outputB['scores'],
                'descriptors0': sp_outputA['descriptors'],
                'descriptors1': sp_outputB['descriptors'],
            })

            for k in pred:
                if isinstance(pred[k], torch.Tensor):
                    pred[k] = pred[k].to(device)

            loss = compute_loss_match(pred)
            if train:
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

            total_loss += loss.item()
            loop.set_postfix(loss=loss.item())

    return total_loss / len(dataloader)



if __name__ == '__main__':
    csv_path = '/kaggle/input/image-matching-challenge-2023/train/train_labels.csv'
    root_dir = '/kaggle/input/image-matching-challenge-2023/train'

    dataset = ImagePairDataset(csv_path, root_dir)
    train_size = int(0.8 * len(dataset))
    val_size = len(dataset) - train_size
    train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

    train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True, pin_memory=True)
    val_loader = DataLoader(val_dataset, batch_size=1, shuffle=False, pin_memory=True)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    model = SuperPoint({})
    model = model.to(device)
    model.load_state_dict(torch.load('/kaggle/input/myfinaldataset/weights/superpoint_v1.pth'))


    for param in matcher.superglue.parameters():
        param.requires_grad = False
    matcher.superglue.eval()
    matcher.superpoint = model
    matcher.superpoint = matcher.superpoint.to(device)
    matcher.superglue = matcher.superglue.to(device)

    optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=15, gamma=0.5)
    best_val_loss = float('inf')

    for epoch in range(100):
        print(f"Epoch {epoch+1}")
        train_loss = run_epoch(model, train_loader, optimizer, device, train=True)
        val_loss = run_epoch(model, val_loader, optimizer, device, train=False)

        print(f"Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f}")

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            torch.save(model.state_dict(), f"superpoint_best.pth")
            print("Saved best model.")
        scheduler.step()

Loaded SuperPoint model
Loaded SuperGlue model ("outdoor" weights)
Loaded SuperPoint model
Epoch 1


Train Loss: 0.5735 | Val Loss: 0.6948
Saved best model.
Epoch 2


Train Loss: 0.7488 | Val Loss: 0.7239
Epoch 3


Train Loss: 0.6823 | Val Loss: 0.6732
Saved best model.
Epoch 4


Train Loss: 0.6697 | Val Loss: 0.6321
Saved best model.
Epoch 5


...

Train Loss: 0.1093 | Val Loss: 0.1176
Epoch 97


Train Loss: 0.1074 | Val Loss: 0.1256
Epoch 98


Train Loss: 0.1311 | Val Loss: 0.2469
Epoch 99


Train Loss: 0.1088 | Val Loss: 0.1284
Epoch 100


Train Loss: 0.1144 | Val Loss: 0.1023


In [3]:
original_sd = torch.load('/kaggle/input/myfinaldataset/weights/superpoint_v1.pth', map_location='cpu')
finetuned_sd = torch.load('superpoint_best.pth', map_location='cpu')

keys = set(finetuned_sd.keys())
ori_keys = set(original_sd.keys())
print("ori_keys:", ori_keys)
print("keys:", keys)



ori_keys: {'conv4a.weight', 'convDa.weight', 'conv1a.weight', 'convPb.bias', 'conv2a.weight', 'convPb.weight', 'conv1a.bias', 'convDa.bias', 'conv1b.weight', 'conv4b.weight', 'conv3a.bias', 'convPa.bias', 'conv1b.bias', 'conv3b.weight', 'convDb.weight', 'conv2a.bias', 'conv3b.bias', 'convDb.bias', 'conv2b.weight', 'conv4b.bias', 'convPa.weight', 'conv3a.weight', 'conv4a.bias', 'conv2b.bias'}
keys: {'conv4a.weight', 'convDa.weight', 'conv1a.weight', 'convPb.bias', 'conv2a.weight', 'convPb.weight', 'conv1a.bias', 'convDa.bias', 'conv1b.weight', 'conv4b.weight', 'conv3a.bias', 'convPa.bias', 'conv1b.bias', 'conv3b.weight', 'convDb.weight', 'conv2a.bias', 'conv3b.bias', 'convDb.bias', 'conv2b.weight', 'conv4b.bias', 'convPa.weight', 'conv3a.weight', 'conv4a.bias', 'conv2b.bias'}
